# Importing Data

## Loading and Displaying DataFrame



Importing the necessary libraries

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import warnings
warnings.filterwarnings("ignore")

Reading the CSV file into a dataFrame and displaying the first few rows of the DataFrame to examine its structure and contents

In [9]:
df = pd.read_csv('Initial_cleaned_df.csv')
pd.set_option('display.max_columns', None)
df.head()

,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
0,118,68.0,22.732803,0,Caucasian,M,180.3,Floor,Floor,92,admit,CTICU,0.541667,73.9,2.3,113.0,502.01,0,0,0.4,31.0,2.51,NaN,3.0,6.0,0.0,4.0,168.0,118.0,27.4,0,40.0,NaN,NaN,NaN,36.0,134.0,39.3,NaN,0,14.1,46.0,32.0,68.0,37.0,68.0,37.0,119.0,72.0,66.0,40.0,89.0,46.0,89.0,46.0,34.0,10.0,100.0,74.0,122.0,64.0,131.0,73.0,131.0,73.0,39.9,37.2,NaN,NaN,68.0,63.0,68.0,63.0,119.0,108.0,NaN,NaN,86.0,85.0,86.0,85.0,26.0,18.0,100.0,74.0,NaN,NaN,131.0,115.0,131.0,115.0,39.5,37.5,2.3,2.3,0.4,0.4,31.0,30.0,8.5,7.4,2.51,2.23,168.0,109.0,19.0,15.0,8.9,8.9,27.4,27.4,NaN,NaN,1.3,1.0,233.0,233.0,4.0,3.4,136.0,134.0,14.1,14.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,1
1,81,77.0,27.421875,0,Caucasian,F,160.0,Floor,Floor,90,admit,Med-Surg ICU,0.927778,70.2,NaN,108.0,203.01,0,0,NaN,9.0,0.56,1.0,1.0,3.0,0.0,1.0,145.0,120.0,36.9,0,46.0,37.0,51.0,7.45,33.0,145.0,35.1,NaN,1,12.7,NaN,NaN,95.0,31.0,95.0,31.0,118.0,72.0,NaN,NaN,120.0,38.0,120.0,38.0,32.0,12.0,100.0,70.0,NaN,NaN,159.0,67.0,159.0,67.0,36.3,35.1,NaN,NaN,61.0,48.0,61.0,48.0,114.0,100.0,Na

Checking the dimensions of the dataframe

In [10]:
df.shape

(130127, 175)

# Data Cleaning Part 2

Examining and removing the rows where 'age,' 'bmi,' and 'glucose_apache' are missing. These columns are vital for assessing diabetes diagnosis in patients. However, these 46 records have so much missing data in remaining columns as well. Therefore, we're dropping these rows to maintain data integrity and ensure accurate analysis.

In [11]:
display(HTML(df[df.age.isna() & df.bmi.isna() & df.glucose_apache.isna()].to_html()))
print('Number of rows with missing values for age, BMI, and glucose_apache (Before dropping):', len(df[df.age.isna() & df.bmi.isna() & df.glucose_apache.isna()]))

missingrow_index = df[df.age.isna() & df.bmi.isna() & df.glucose_apache.isna()].index

print('Total dataframe size (Before dropping): ', len(df))
df.drop(missingrow_index, inplace=True)
print('Total dataframe size (After dropping): ', len(df))

,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
1310,118,NaN,NaN,0,Caucasian,F,160.0,Emergency Department,Accident & Emergency,100,admit,Neuro ICU,0.019444,NaN,NaN,301.0,403.01,0,0,NaN,NaN,NaN,NaN,4.0,6.0,0.0,2.0,NaN,110.0,NaN,0,152.0,NaN,NaN,NaN,45.0,NaN,36.1,NaN,0,NaN,NaN,NaN,109.0,66.0,109.0,66.0,104.0,64.0,NaN,NaN,152.0,67.0,152.0,67.0,52.0,13.0,100.0,96.0,NaN,NaN,176.0,114.0,176.0,114.00,37.700000,36.100,NaN,NaN,105.0,93.0,105.0,93.0,94.0,74.0,NaN,NaN,152.0,110.0,152.0,110.0,45.0,26.0,99.0,97.0,NaN,NaN,181.0,144.0,181.0,144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.0,83.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
2671,77,NaN,NaN,0,Caucasian,F,162.6,Emergency Department,Accident & Emergency,87,admit,Med-Surg ICU,0.054167,NaN,NaN,113.0,502.01,0,0,NaN,NaN,NaN,NaN,4.0,6.0,0.0,5.0,NaN,100.0,NaN,0,61.0,NaN,NaN,NaN,41.0,NaN,36.3,NaN,0,NaN,NaN,NaN,69.0,47.0,69.0,47.0,94.0,65.0,NaN,NaN,99.0,61.0,99.0,61.0,34.0,14.0,100.0,94.0,NaN,NaN,156.0,87.0,156.0,87.00,37.500000,36.300,NaN,NaN,56.

Number of rows with missing values for age, BMI, and glucose_apache (Before dropping): 46
Total dataframe size (Before dropping):  130127
Total dataframe size (After dropping):  130081


# Handling Missing values

Verifying the percentage of missing values in each column

In [12]:
column_missing_info = []
for col in df.columns:
  if(df[col].isnull().any()):
    column_missing_info.append({
        'Column': col,
        'Data Type': df[col].dtype,
        'Null values count': df[col].isnull().sum(),
        '% of missing data': ((df[col].isnull().sum())/len(df))*100
    })

if column_missing_info:
  column_info_df = pd.DataFrame(column_missing_info)
  display(HTML(column_info_df.sort_values(by=["Data Type", "Null values count"]).to_html()))
else:
  print('No missing values!')

,Column,Data Type,Null values count,% of missing data
37,d1_heartrate_max,float64,241,0.185269
38,d1_heartrate_min,float64,241,0.185269
51,d1_sysbp_max,float64,249,0.191419
52,d1_sysbp_min,float64,249,0.191419
33,d1_diasbp_max,float64,255,0.196032
34,d1_diasbp_min,float64,255,0.196032
20,heart_rate_apache,float64,288,0.221401
41,d1_mbp_max,float64,306,0.235238
42,d1_mbp_min,float64,306,0.235238
22,map_apache,float64,398,0.305963


From the above output, It is clear that over 70 columns have more than 50% missing values. Handling such a huge number of columns with considerable missing data is difficult, and removing all columns might result in major data loss. To minimize this, we've set a 75% threshold and will eliminate columns that surpass it, establishing a balance between data preservation and data quality.

In [13]:
print('Number of columns in the dataframe before dropping: ', len(df.columns))

#Removing columns with more than 75% missing values
missing_value_columns = column_info_df[column_info_df['% of missing data'] > 75]['Column'].tolist()
df = df.drop(missing_value_columns, axis=1)

print('Number of columns we are dropping:' , len(missing_value_columns))
print('Number of columns in the dataframe after dropping: ', len(df.columns))

Number of columns in the dataframe before dropping:  175
Number of columns we are dropping: 44
Number of columns in the dataframe after dropping:  131


Re-assigning columns after removing few columns to their respective variables

In [15]:
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = df.select_dtypes(include=['object']).columns

*Filling missing values in categorical columns with the most frequently occurring value*

In [16]:
for col in df.columns:
    if df[col].dtype == 'object' and df[col].isnull().any():
        df[col] = df.groupby('diabetes_mellitus')[col].transform(lambda x: x.fillna(x.mode()[0]))

*Filling missing values in numerical columns by using KNN Imputer*

In [5]:
# Impute numeric columns using KNN
numerical_imputer = KNNImputer(n_neighbors=5, weights='distance')
from tqdm import tqdm

for col in tqdm(numeric_cols, desc="Imputing columns"):
    df[col] = numerical_imputer.fit_transform(df[[col]])

Imputing columns: 100%|██████████| 125/125 [4:34:42<00:00, 131.86s/it]  


In [6]:
column_missing_info = []
for col in df.columns:
  if(df[col].isnull().any()):
    column_missing_info.append({
        'Column': col,
        'Data Type': df[col].dtype,
        'Null values count': df[col].isnull().sum(),
        '% of missing data': ((df[col].isnull().sum())/len(df))*100
    })

if column_missing_info:
  column_info_df = pd.DataFrame(column_missing_info)
  display(HTML(column_info_df.sort_values(by=["Data Type", "Null values count"]).to_html()))
else:
  print('No missing values!')

No missing values!


As we can see, we filled in the missing values. Now we can proceed to the next steps.

In [7]:
# Save the imputed dataset
df.to_csv('imputed_dataset.csv', index=False)